In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("tarun")

In [3]:
!pip uninstall -y peft

Found existing installation: peft 0.14.0
Uninstalling peft-0.14.0:
  Successfully uninstalled peft-0.14.0


In [4]:
!pip uninstall -y transformers
!pip install --no-cache-dir transformers==4.31.0 accelerate

Found existing installation: transformers 4.51.1
Uninstalling transformers-4.51.1:
  Successfully uninstalled transformers-4.51.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 133.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 341.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 282.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 101.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 129.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 137.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 152.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 166.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 167.4 MB/s eta 0:00

In [4]:
!pip install --no-cache-dir -q \
      "evaluate @ git+https://github.com/huggingface/evaluate.git"


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [5]:
# ⬇︎ 1-A  UNINSTALL the old package, then
# ⬇︎ 1-B  INSTALL the latest code straight from GitHub
!pip uninstall -y evaluate
!pip install --no-cache-dir -q \
      "evaluate @ git+https://github.com/huggingface/evaluate.git"

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [6]:
!mkdir -p local_metrics/cider
!wget -q https://raw.githubusercontent.com/huggingface/evaluate/main/metrics/cider/cider.py \
         -O  local_metrics/cider/cider.py
!wget -q https://raw.githubusercontent.com/huggingface/evaluate/main/metrics/cider/__init__.py \
         -O  local_metrics/cider/__init__.py

In [7]:
!pip install -q --no-cache-dir pycocoevalcap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 320.3 MB/s eta 0:00:00a 0:00:01


In [8]:
from pycocoevalcap.cider.cider import Cider as CiderScorer

def cider_score(preds, refs):
    """
    preds : List[str]            – one generated caption per image
    refs  : List[List[str]]      – 5 (or more) ground-truth captions per image
    returns a single float in [0, 100]
    """
    scorer = CiderScorer()
    gts, res = {}, {}

    for i, (pred, ref_list) in enumerate(zip(preds, refs)):
        res[i] = [pred]           # list *of strings* (hypotheses)
        gts[i] = ref_list         # list *of strings* (references)

    score, _ = scorer.compute_score(gts, res)
    return score

In [9]:
p  = ["a cat on a mat"]
rs = [["a cat is sitting on the mat", "cat on a mat"]]
print("CIDEr:", cider_score(p, rs))       # e.g. CIDEr: 41.2

CIDEr: 0.0


In [10]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b695efe123d1afcf70ebf25e164ae86162b94781c83d449b6780f5df32c73a76
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


# Without Fine tuning

In [13]:
#####################################################################
# 0.  Runtime installs  (run FIRST, before any other import)
#####################################################################
# !pip install -q --no-cache-dir \
#       "transformers==4.35.2" "accelerate>=0.25.0" "evaluate>=0.4.3" \
#       "sacrebleu" "rouge_score"

#####################################################################
# 1.  Imports & basic setup
#####################################################################
import os, random, ssl, math, numpy as np, json, torch
from pathlib import Path
from collections import defaultdict
from PIL import Image
from io import BytesIO
from tqdm.auto import tqdm
from typing import List

from torchvision import transforms
from torch.utils.data import Dataset, random_split
from torch import nn

from transformers import (
    VisionEncoderDecoderModel, ViTImageProcessor,
    AutoTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    get_cosine_schedule_with_warmup,
    default_data_collator, set_seed
)
# ---- internal GPT-2 building block
from transformers.models.gpt2.modeling_gpt2 import GPT2Block

# from transformers import (
#     VisionEncoderDecoderModel, ViTImageProcessor,
#     AutoTokenizer, GPT2Block,
#     Seq2SeqTrainingArguments, Seq2SeqTrainer,
#     get_cosine_schedule_with_warmup,
#     default_data_collator, set_seed
# )
from evaluate import load

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(42)
ssl._create_default_https_context = ssl._create_unverified_context

#####################################################################
# 2.  Paths & hyper-parameters
#####################################################################
DATASET_DIR   = Path("/kaggle/input/flickr8k")        # ← change if needed
IMG_DIR       = DATASET_DIR / "Images"
CAPTION_FILE  = DATASET_DIR / "captions.txt"

ENCODER_NAME  = "google/vit-base-patch16-224-in21k"
DECODER_NAME  = "gpt2"

EPOCHS                 = 10
TRAIN_VAL_SPLIT        = 0.95
PER_DEVICE_BATCH       = 2
GRAD_ACCUM_STEPS       = 8           # effective 16
LR_DECODER             = 3e-5        # a bit lower because we add fresh blocks
LR_ENCODER             = 1e-5
MAX_SEQ_LEN            = 24
WARMUP_RATIO           = 0.05
WEIGHT_DECAY           = 0.01
LABEL_SMOOTH           = 0.1
NEW_GPT2_BLOCKS        = 2           # ← extra capacity
UNFREEZE_VIT_AFTER_EP  = 2
UNFREEZE_VIT_LAST_N    = 2           # fine-tune last 2 encoder blocks

#####################################################################
# 3.  Caption loader (comma-separated Flickr8k)
#####################################################################
import pandas as pd
def load_flickr(path: Path):
    df = pd.read_csv(path)
    buckets = defaultdict(list)
    for img, cap in zip(df["image"], df["caption"]):
        buckets[img.split("#")[0]].append(cap.strip())
    return list(buckets.items())

items = load_flickr(CAPTION_FILE)
print("Unique images:", len(items))

#####################################################################
# 4.  Transforms, processor, tokenizer
#####################################################################
image_processor = ViTImageProcessor.from_pretrained(ENCODER_NAME)
tokenizer = AutoTokenizer.from_pretrained(DECODER_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

train_tfm = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.9,1.0)),
    transforms.RandomHorizontalFlip()
])
val_tfm = transforms.Resize((224,224))

#####################################################################
# 5.  Dataset
#####################################################################
class ImgCapDS(Dataset):
    def __init__(self, pairs, img_root, tfm):
        self.pairs = pairs; self.root = img_root; self.tfm = tfm
    def __len__(self): return len(self.pairs)
    def __getitem__(self, idx):
        img_name, caps = self.pairs[idx]
        with Image.open(self.root/img_name).convert("RGB") as im:
            im = self.tfm(im)
        px = image_processor(im, return_tensors="pt").pixel_values.squeeze(0)
        cap = random.choice(caps)
        lab = tokenizer(cap, max_length=MAX_SEQ_LEN,
                        padding="max_length", truncation=True).input_ids
        lab = [(x if x!=tokenizer.pad_token_id else -100) for x in lab]
        return {"pixel_values": px,
                "labels": torch.tensor(lab, dtype=torch.long),
                "all_caps": caps}

full_train = ImgCapDS(items, IMG_DIR, train_tfm)
full_val   = ImgCapDS(items, IMG_DIR, val_tfm)
n_train    = int(TRAIN_VAL_SPLIT*len(full_train))
train_ds,_ = random_split(full_train,[n_train,len(full_train)-n_train])
val_ds     = random_split(full_val  ,[n_train,len(full_val)-n_train])[1]
print(f"train {len(train_ds)}  | val {len(val_ds)}")

# 6.  Build base ViT-GPT-2, then **extend** it
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
            ENCODER_NAME, DECODER_NAME, tie_encoder_decoder=False)

dec_conf  = model.decoder.config
enc_size  = model.encoder.config.hidden_size
dec_size  = dec_conf.n_embd

# 6-A  2-layer MLP between encoder and decoder
model.enc_to_dec_proj = nn.Sequential(
    nn.Linear(enc_size, dec_size),
    nn.Tanh(),
    nn.Dropout(0.1),
    nn.Linear(dec_size, dec_size),
).to(device)

# 6-B  Append fresh GPT-2 blocks
for _ in range(NEW_GPT2_BLOCKS):
    model.decoder.transformer.h.append(GPT2Block(dec_conf).to(device))

new_n = len(model.decoder.transformer.h)          # ← 14
dec_conf.n_layer = new_n 
model.decoder.transformer.n_layer = new_n        # NEW: internal attribute
model.decoder.transformer.config.n_layer = new_n # NEW: its own config ref
model.decoder.config.n_layer = new_n

# 6-C  misc config …

# ---- 6-C  Misc config tweaks ----
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id           = tokenizer.eos_token_id
model.config.pad_token_id           = tokenizer.pad_token_id
model.config.max_length             = MAX_SEQ_LEN
model.config.no_repeat_ngram_size   = 4
model.config.length_penalty         = 1.2
model.config.early_stopping         = True
model.config.label_smoothing        = LABEL_SMOOTH

# Freeze encoder at start
for p in model.encoder.parameters(): p.requires_grad = False
model.to(device)

#####################################################################
# 7.  Metrics (BLEU & ROUGE-L)
#####################################################################
bleu = load("bleu"); rouge = load("rouge")
def _post(preds, labels):
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return [p.strip() for p in preds], [[l.strip()] for l in labels]

def compute_metrics(eval_pred):
    p,l = _post(*eval_pred)
    out = bleu.compute(predictions=p, references=l)
    out["rougeL"] = rouge.compute(predictions=p,
                     references=[r[0] for r in l],
                     rouge_types=["rougeL"])["rougeL"]
    out["cider"]  = cider_score(p, l)
    return out

#####################################################################
# 8.  Custom Trainer: 2-group optimiser + gradual unfreeze
#####################################################################
class VCaptionTrainer(Seq2SeqTrainer):
    def _setup_training(self,*a,**k):
        super()._setup_training(*a,**k)
        enc,dec=[],[]
        for n,p in self.model.named_parameters():
            (enc if n.startswith("encoder.") else dec).append(p)
        self.optimizer = torch.optim.AdamW(
            [{"params":enc,"lr":LR_ENCODER},
             {"params":dec,"lr":LR_DECODER}], lr=LR_DECODER,
            weight_decay=WEIGHT_DECAY)
        total = self.args.max_steps
        warm  = int(WARMUP_RATIO*total)
        self.lr_scheduler = get_cosine_schedule_with_warmup(
            self.optimizer, warm, total)
    def training_step(self,*a,**k):
        if self.state.epoch and self.state.epoch>=UNFREEZE_VIT_AFTER_EP:
            for idx in range(12-UNFREEZE_VIT_LAST_N,12):
                for p in self.model.encoder.encoder.layer[idx].parameters():
                    p.requires_grad=True
        return super().training_step(*a,**k)

#####################################################################
# 9.  TrainingArguments
#####################################################################
args = Seq2SeqTrainingArguments(
    output_dir="vit_gpt2_extended",
    per_device_train_batch_size=PER_DEVICE_BATCH,
    per_device_eval_batch_size=PER_DEVICE_BATCH,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    generation_max_length=MAX_SEQ_LEN,
    generation_num_beams=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="cider",
    greater_is_better=True,
    fp16=True,
    logging_steps=50,
    report_to="none",
)

trainer = VCaptionTrainer(
    model=model, args=args,
    train_dataset=train_ds, eval_dataset=val_ds,
    tokenizer=image_processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

#####################################################################
# 10.  Train & evaluate
#####################################################################
trainer.train()
trainer.evaluate()

# #####################################################################
# # 11.  Inference helper  (unchanged API)
# #####################################################################
# import requests, textwrap
# def caption_url(url, num_beams=4):
#     model.eval()
#     img = Image.open(BytesIO(requests.get(url,timeout=10).content)).convert("RGB")
#     px  = image_processor(img, return_tensors="pt").pixel_values.to(device)
#     with torch.no_grad():
#         out = model.generate(px, max_length=MAX_SEQ_LEN, num_beams=num_beams,
#                              eos_token_id=tokenizer.eos_token_id,
#                              pad_token_id=tokenizer.pad_token_id)
#     return textwrap.shorten(tokenizer.decode(out[0], skip_special_tokens=True), 120)

# # caption_url("https://huggingface.co/datasets/nateraw/horses-or-humans/resolve/main/train/horse/horse01.png")


2025-04-27 03:45:39.656506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745725539.850448      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745725539.903688      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Unique images: 8091


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length,Rougel,Cider
0,2.848100,2.699446,0.033205,"[0.19982595453062113, 0.04676832043695949, 0.017893355600620303, 0.0072699924793181245]",1.000000,1.965156,9193,4678,0.233937,0.157716
1,2.638800,2.500588,0.036753,"[0.20729720925181888, 0.0528169014084507, 0.019883319442790807, 0.008381285964473356]",1.000000,1.972793,9209,4668,0.248284,0.207218
2,2.597500,2.543770,0.028674,"[0.1949134199134199, 0.04470854555744199, 0.015183867141162514, 0.005109034267912773]",1.000000,1.996111,9240,4629,0.237741,0.155776
3,2.639700,2.562127,0.032096,"[0.2045060658578856, 0.046788263283108644, 0.017098076466397532, 0.006486216789322689]",1.000000,1.895299,9232,4871,0.245902,0.180522
4,2.635000,2.536200,0.031799,"[0.2091411242283115, 0.05018124150430449, 0.016621156357592308, 0.0058618109254178095]",1.000000,1.918745,9233,4812,0.251728,0.184182
5,2.583700,2.531303,0.034483,"[0.20753899480069324, 0.05075337034099921, 0.01887912609831394, 0.007109891480603717]",1.000000,1.924937,9232,4796,0.251029,0.187008
6,2.582400,2.613423,0.031449,"[0.2019501625135428, 0.050538243626062324, 0.017458432304038006, 0.005489706799750468]",1.000000,1.964248,9230,4699,0.248362,0.167334
7,2.590700,2.547881,0.036108,"[0.21053772766695578, 0.055221680462637486, 0.019847872593296886, 0.007366712448495442]",1.000000,1.930515,9224,4778,0.256353,0.213324


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


{'eval_loss': 2.5550448894500732,
 'eval_bleu': 0.03435276787915125,
 'eval_precisions': [0.2032740676496097,
  0.049325320331103296,
  0.018540527691942,
  0.007491571981520789],
 'eval_brevity_penalty': 1.0,
 'eval_length_ratio': 1.9751605995717345,
 'eval_translation_length': 9224,
 'eval_reference_length': 4670,
 'eval_rougeL': 0.2501312044806918,
 'eval_cider': 0.1968407337684648,
 'eval_runtime': 152.3178,
 'eval_samples_per_second': 2.659,
 'eval_steps_per_second': 1.333,
 'epoch': 7.99}

In [15]:
# 11.  Inference helper  (unchanged API)
#####################################################################
import requests, textwrap
def caption_url(url, num_beams=4):
    model.eval()
    img = Image.open(BytesIO(requests.get(url,timeout=10).content)).convert("RGB")
    px  = image_processor(img, return_tensors="pt").pixel_values.to(device)
    with torch.no_grad():
        out = model.generate(px, max_length=MAX_SEQ_LEN, num_beams=num_beams,
                             eos_token_id=tokenizer.eos_token_id,
                             pad_token_id=tokenizer.pad_token_id)
    return textwrap.shorten(tokenizer.decode(out[0], skip_special_tokens=True), 120)

caption_url("https://images.pexels.com/photos/1054655/pexels-photo-1054655.jpeg?cs=srgb&dl=pexels-hsapir-1054655.jpg&fm=jpg")


'A man in a blue shirt is standing on the shore of a lake.... a man in a white'

# Fine tuned version - VIT GPT 2

In [ ]:
#####################################################################
# 0.  One-time installs – run BEFORE any other import
#####################################################################
# !pip install -q --no-cache-dir "transformers==4.35.2" "accelerate>=0.25.0" \
#                            "evaluate>=0.4.3" sacrebleu rouge_score \
#                            pycocoevalcap

#####################################################################
# 1.  Imports & basic setup
#####################################################################
import os, random, ssl, math, json, torch, numpy as np, pandas as pd
from pathlib import Path
from collections import defaultdict
from typing import List
from PIL import Image
from io import BytesIO
from tqdm.auto import tqdm

from torchvision import transforms
from torch.utils.data import Dataset, random_split, WeightedRandomSampler
from torch import nn

from transformers import (
    VisionEncoderDecoderModel, ViTImageProcessor,
    AutoTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    get_cosine_schedule_with_warmup,
    default_data_collator, set_seed
)
from transformers.models.gpt2.modeling_gpt2 import GPT2Block

from evaluate import load
from pycocoevalcap.cider.cider import Cider as CiderScorer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(42)
ssl._create_default_https_context = ssl._create_unverified_context

#####################################################################
# 2.  Paths & hyper-parameters
#####################################################################
DATASET_DIR   = Path("/kaggle/input/flickr8k")        # change if needed
IMG_DIR       = DATASET_DIR / "Images"
CAPTION_FILE  = DATASET_DIR / "captions.txt"

ENCODER_NAME  = "google/vit-base-patch16-224-in21k"
DECODER_NAME  = "gpt2"

EPOCHS                 = 8
TRAIN_VAL_SPLIT        = 0.95
PER_DEVICE_BATCH       = 2
GRAD_ACCUM_STEPS       = 16      # effective 32
LR_DECODER             = 2e-5
LR_ENCODER             = 5e-6
MAX_SEQ_LEN            = 20
WARMUP_RATIO           = 0.1
WEIGHT_DECAY           = 0.01
LABEL_SMOOTH           = 0.1
NEW_GPT2_BLOCKS        = 2
UNFREEZE_VIT_AFTER_EP  = 2
UNFREEZE_VIT_LAST_N    = 12      # unfreeze all blocks

#####################################################################
# 3.  Load captions & expand to 1-caption-per-row
#####################################################################
df = pd.read_csv(CAPTION_FILE)
pairs = defaultdict(list)
for img, cap in zip(df["image"], df["caption"]):
    pairs[img.split("#")[0]].append(cap.strip())

expanded = [(img, cap) for img, caps in pairs.items() for cap in caps]
print("Unique images:", len(pairs), "  expanded rows:", len(expanded))

#####################################################################
# 4.  Processor, tokenizer, transforms
#####################################################################
processor = ViTImageProcessor.from_pretrained(ENCODER_NAME)
tokenizer = AutoTokenizer.from_pretrained(DECODER_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.RandomHorizontalFlip()
])
val_tf = transforms.Resize((224, 224))

#####################################################################
# 5.  Dataset with length-aware weights
#####################################################################
class FlickrDS(Dataset):
    def __init__(self, rows, img_root, tfm):
        self.rows, self.root, self.tfm = rows, img_root, tfm
        self.weights = torch.tensor([min(1.0, 10/len(c.split())) for _, c in rows])

    def __len__(self): return len(self.rows)

    def __getitem__(self, idx):
        img_name, cap = self.rows[idx]
        with Image.open(self.root/img_name).convert("RGB") as im:
            im = self.tfm(im)
        px = processor(im, return_tensors="pt").pixel_values.squeeze(0)
        lab = tokenizer(cap, max_length=MAX_SEQ_LEN,
                        padding="max_length", truncation=True).input_ids
        lab = [(x if x!=tokenizer.pad_token_id else -100) for x in lab]
        return {"pixel_values": px, "labels": torch.tensor(lab, dtype=torch.long),
                "img": img_name}

full_ds   = FlickrDS(expanded, IMG_DIR, train_tf)
val_full  = FlickrDS(expanded, IMG_DIR, val_tf)

n_train   = int(TRAIN_VAL_SPLIT*len(full_ds))
train_ds, _ = random_split(full_ds, [n_train, len(full_ds)-n_train])
val_ds      = random_split(val_full, [n_train, len(full_ds)-n_train])[1]
print(f"train {len(train_ds)} | val {len(val_ds)}")

#####################################################################
# 6.  Build ViT-GPT-2 & extend decoder
#####################################################################
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
            ENCODER_NAME, DECODER_NAME, tie_encoder_decoder=False)

# 6-A  2-layer MLP bridge
hid_e = model.encoder.config.hidden_size
hid_d = model.decoder.config.n_embd
model.enc_to_dec_proj = nn.Sequential(
    nn.Linear(hid_e, hid_d),
    nn.Tanh(),
    nn.Dropout(0.1),
    nn.Linear(hid_d, hid_d),
).to(device)

# 6-B  add GPT-2 blocks
for _ in range(NEW_GPT2_BLOCKS):
    model.decoder.transformer.h.append(GPT2Block(model.decoder.config).to(device))
new_layers = len(model.decoder.transformer.h)
for obj in (model.decoder.transformer,
            model.decoder.transformer.config,
            model.decoder.config):
    obj.n_layer = new_layers

# 6-C  generation + smoothing
cfg = model.config
cfg.decoder_start_token_id = tokenizer.bos_token_id
cfg.eos_token_id           = tokenizer.eos_token_id
cfg.pad_token_id           = tokenizer.pad_token_id
cfg.max_length             = MAX_SEQ_LEN
cfg.no_repeat_ngram_size   = 5
cfg.length_penalty         = 1.6
cfg.early_stopping         = True
cfg.label_smoothing        = LABEL_SMOOTH
cfg.teacher_forcing_ratio  = 1.0   # start full TF

# freeze encoder initially
for p in model.encoder.parameters(): p.requires_grad = False
model.to(device)

#####################################################################
# 7.  Metrics – BLEU, ROUGE-L, CIDEr
#####################################################################
bleu   = load("bleu")
rouge  = load("rouge")
ciderS = CiderScorer()

def postproc(preds, labels):
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return [p.strip() for p in preds], [[l.strip()] for l in labels]

def cider_score(preds, refs):
    scorer = CiderScorer()
    gts, res = {}, {}
    for i,(p,r) in enumerate(zip(preds, refs)):
        gts[i] = [x for x in r]
        res[i] = [p]
    sc,_ = scorer.compute_score(gts, res)
    return sc

def compute_metrics(eval_pred):
    p,l = postproc(*eval_pred)
    out = bleu.compute(predictions=p, references=l)
    out["rougeL"] = rouge.compute(predictions=p,
                     references=[r[0] for r in l],
                     rouge_types=["rougeL"])["rougeL"]
    out["cider"]  = cider_score(p, l)
    return out

#####################################################################
# 8.  Custom Trainer (2-group opt, gradual unfreeze, scheduled sampling)
#####################################################################
class VCaptionTrainer(Seq2SeqTrainer):
    def _setup_training(self,*a,**k):
        super()._setup_training(*a,**k)
        enc, dec = [], []
        for n,p in self.model.named_parameters():
            (enc if n.startswith("encoder.") else dec).append(p)
        self.optimizer = torch.optim.AdamW(
            [{"params":enc,"lr":LR_ENCODER},
             {"params":dec,"lr":LR_DECODER}],
             lr=LR_DECODER, weight_decay=WEIGHT_DECAY)
        warm = int(WARMUP_RATIO*self.args.max_steps)
        self.lr_scheduler = get_cosine_schedule_with_warmup(
            self.optimizer, warm, self.args.max_steps)

    def get_train_dataloader(self):
        # length-aware sampler
        sampler = WeightedRandomSampler(
            weights=self.train_dataset.dataset.weights[self.train_dataset.indices],
            num_samples=len(self.train_dataset), replacement=True)
        return torch.utils.data.DataLoader(
            self.train_dataset, batch_size=self.args.per_device_train_batch_size,
            sampler=sampler, collate_fn=default_data_collator)

    def training_step(self, model, inputs):
        # unfreeze ViT blocks gradually
        if self.state.epoch is not None and self.state.epoch >= UNFREEZE_VIT_AFTER_EP:
            for idx in range(12-UNFREEZE_VIT_LAST_N, 12):
                for p in model.encoder.encoder.layer[idx].parameters():
                    p.requires_grad = True
            for g in self.optimizer.param_groups:
                if g["lr"] == LR_ENCODER:
                    g["lr"] = LR_ENCODER * 0.3      # e.g. 1.5 e-6
        # linear teacher-forcing decay to 0.75
        tf_ratio = max(0.75, 1.0 - 0.25*(self.state.epoch/(EPOCHS-1)))
        model.config.teacher_forcing_ratio = tf_ratio
        return super().training_step(model, inputs)

#####################################################################
# 9.  TrainingArguments
#####################################################################
args = Seq2SeqTrainingArguments(
    output_dir="vitgpt2_plus",
    per_device_train_batch_size=PER_DEVICE_BATCH,
    per_device_eval_batch_size=PER_DEVICE_BATCH,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    generation_max_length=MAX_SEQ_LEN,
    generation_num_beams=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit = 1,
    metric_for_best_model="cider",
    greater_is_better=True,
    fp16=True,
    fp16_full_eval=True,          # NEW – eval in fp16 avoids cast-mismatch
    max_grad_norm=1.0,            # NEW – clip every backward() call
    gradient_checkpointing=True,  # NEW – saves RAM *and* keeps grad norms small
    logging_steps=50,
    report_to="none",
)

trainer = VCaptionTrainer(
    model=model, args=args,
    train_dataset=train_ds, eval_dataset=val_ds,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics)

#####################################################################
# 10.  Train & evaluate
#####################################################################
trainer.train()
trainer.evaluate()

#####################################################################
# 11.  Quick inference helper
#####################################################################
# def caption_url(url, beams=4):
#     model.eval()
#     import requests, textwrap
#     im = Image.open(BytesIO(requests.get(url, timeout=10).content)).convert("RGB")
#     px = processor(im, return_tensors="pt").pixel_values.to(device)
#     with torch.no_grad():
#         out = model.generate(px, max_length=MAX_SEQ_LEN, num_beams=beams,
#                              eos_token_id=tokenizer.eos_token_id,
#                              pad_token_id=tokenizer.pad_token_id)
#     return textwrap.shorten(tokenizer.decode(out[0], skip_special_tokens=True), 120)

# Example:
# print(caption_url("https://farm4.staticflickr.com/3629/3336631456_09f313e363.jpg"))


Unique images: 8091   expanded rows: 40455


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


train 38432 | val 2023


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.4.crossattention.c_attn.bias', 'h.4.crossattention.c_proj.weight', 'h.0.crossattention.c_proj.bias', 'h.9.crossattention.c_attn.bias', 'h.11.ln_cross_attn.bias', 'h.4.ln_cross_attn.bias', 'h.11.crossattention.c_attn.bias', 'h.10.crossattention.c_attn.weight', 'h.11.crossattention.c_proj.bias', 'h.7.crossattention.c_attn.bias', 'h.4.crossattention.c_proj.bias', 'h.2.crossattention.q_attn.bias', 'h.10.crossattention.c_attn.bias', 'h.2.crossattention.c_attn.bias', 'h.8.crossattention.q_attn.bias', 'h.5.crossattention.c_proj.weight', 'h.6.crossattention.c_attn.bias', 'h.9.crossattention.c_attn.weight', 'h.5.crossattention.q_attn.bias', 'h.3.crossattention.c_proj.weight', 'h.4.crossattention.q_attn.weight', 'h.8.crossattention.c_attn.weight', 'h.10.crossattention.q_attn.weight', 'h.5.crossattention.q_attn.weight', 'h.1.ln_cross_attn.bias', 'h.1.crossattention.c_proj.bias', '

Epoch,Training Loss,Validation Loss,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length,Rougel,Cider
1,2.374500,2.345213,0.046534,"[0.2399133352709594, 0.0653751674854846, 0.026537676400106507, 0.011265655484926678]",1.000000,1.613669,37847,23454,0.269493,0.254084
2,2.174200,2.117774,0.054936,"[0.25891952429203774, 0.07517122805039601, 0.03275944524151124, 0.014284350841472337]",1.000000,1.598960,37502,23454,0.279273,0.299178
3,2.029700,1.980820,0.060161,"[0.2654248190833557, 0.08337348031853091, 0.03676647426647427, 0.016100636983451235]",1.000000,1.590773,37310,23454,0.281816,0.316163
4,1.926900,1.871343,0.063597,"[0.2689347372922123, 0.08665065687243961, 0.038745055735347, 0.01811860027433092]",1.000000,1.595378,37418,23454,0.287508,0.325660
5,1.837700,1.792112,0.068179,"[0.27371908841118475, 0.09091427106185361, 0.041933728383117666, 0.02070653399027469]",1.000000,1.582758,37122,23454,0.290879,0.347218
6,1.796700,1.723977,0.069103,"[0.2740279686363269, 0.09307495012824167, 0.0433664983215895, 0.02061590001288494]",1.000000,1.582374,37113,23454,0.289821,0.339176
7,1.747900,1.681991,0.071028,"[0.2750646830530401, 0.09426755223625324, 0.04440680016939924, 0.02210407604317706]",1.000000,1.581990,37104,23454,0.291821,0.352679


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=Tr